In [12]:
import os
import IPython
from ipywidgets import interact
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [13]:
#get all the path of the file
current_path = os.getcwd()
dev_path = os.path.join(current_path, '10902294')
machine_choice = [i for i in os.listdir(dev_path) if ".DS_Store" not in i]

data_choice = ["train", "test"]
domain_choice = ["source","target"]
condition_choice = ["normal", "anomaly"]

In [14]:
#get unique attribute and the overall look at the dataset
def sub_df_from_name (df:pd.DataFrame,name:str):
    indices = []
    for file in df["file_name"].tolist():
        if name in file:
            idx = df["file_name"].tolist().index(file)
            indices.append(idx)

    return df.iloc[indices]

def check_unique(dev_path, machine_choice):
    name_attributt = "attributes_00.csv"

    print("Overall look at the dataset")
    print("------------------------------------------------------------------------------------------------")
    print()
    
    #machine type
    for m in machine_choice:
        print("Machine: ",m)
        attributt_path = os.path.join(dev_path,m,name_attributt)
        df = pd.read_csv(attributt_path)
        print()

        #data type
        for type in ["train","test"]:
            print("data: ",type)
            sub_df_type = sub_df_from_name(df,type)

            #domain type
            for domain in ["source","target"]:
                sub_df_domain = sub_df_from_name(sub_df_type,domain)
                
                #condition type
                if type == "test":
                    for condition in ["normal","anomaly"]:
                        print("domain condition: {}_{}".format(domain,condition))
                        sub_df_condition = sub_df_from_name(sub_df_domain,condition)

                        for column in sub_df_condition.columns:
                            if column != "file_name":
                                print("Column: ",column,end=", ")
                                unique_values = np.sort(sub_df_condition[column].unique())
                                print("Unique_values:", unique_values)
                else:
                    print("domain ", domain)
                    for column in sub_df_domain.columns:
                        if column != "file_name":
                            print("Column: ",column,end=", ")
                            unique_values = np.sort(sub_df_domain[column].unique())
                            print("Unique_values:", unique_values)
            print()
        print("------------------------------------------------------------------------------------------------")
        print()
check_unique(dev_path,machine_choice)


Overall look at the dataset
------------------------------------------------------------------------------------------------

Machine:  bearing

data:  train
domain  source
Column:  d1p, Unique_values: ['pro']
Column:  d1v, Unique_values: ['A' 'B']
Column:  d1p.1, Unique_values: ['vel']
Column:  d1v.1, Unique_values: [ 4  8 12 16]
Column:  d1p.2, Unique_values: ['loc']
Column:  d1v.2, Unique_values: ['A' 'B' 'C' 'D']
domain  target
Column:  d1p, Unique_values: ['pro']
Column:  d1v, Unique_values: ['A' 'B']
Column:  d1p.1, Unique_values: ['vel']
Column:  d1v.1, Unique_values: [ 4  8 12 16]
Column:  d1p.2, Unique_values: ['loc']
Column:  d1v.2, Unique_values: ['E' 'F' 'G' 'H']

data:  test
domain condition: source_normal
Column:  d1p, Unique_values: ['pro']
Column:  d1v, Unique_values: ['A' 'B']
Column:  d1p.1, Unique_values: ['vel']
Column:  d1v.1, Unique_values: [ 4  8 12 16]
Column:  d1p.2, Unique_values: ['loc']
Column:  d1v.2, Unique_values: ['A' 'B' 'C' 'D']
domain condition: sourc

In [19]:
@interact(machine = machine_choice, data = data_choice, condition = condition_choice, domain = domain_choice)
def load_clip (machine,data,domain, condition):

    #load all clips
    clip_path = os.path.join(dev_path,machine,data)
    clip = [i for i in os.listdir(clip_path) if ".DS_Store" not in i]
    
    #attributtes bearing
    if machine == "bearing":
        pro_choice = ["A","B"]
        vel_choice = [4,8,12,16]
        if (data == "train" and condition == "normal") or (data == "test" and condition == "normal") or (data == "test" and condition == "anomaly") :
            if domain == "source":
                loc_choice = ['A', 'B', 'C', 'D']
            elif domain == "target":
                loc_choice = ['E', 'F', 'G' ,'H']
        else:
            return "No clips are founded"

        
        @interact(pro = pro_choice, vel = vel_choice,loc = loc_choice)
        def bearing_clip(pro, vel, loc):
            
            #key word data, domain, condtion and attributtes
            key_word_data = "{}_{}_{}".format(domain,data,condition)
            key_word_att = "pro_{}_vel_{}_loc_{}".format(pro,vel,loc)
            clip_bearing = [i for i in clip if (key_word_data in i) and (key_word_att in i)]
            clip_indices = [i.split("_")[5] for i in clip_bearing]

            @interact(clip = clip_indices)
            def clip_select(clip):

                #key word index
                key_word = "{}_{}_{}".format(key_word_data,clip,key_word_att)
                selection = [i for i in clip_bearing if key_word in i][0]
                path_selection = os.path.join(clip_path,selection)

                #display the audio clip
                display(IPython.display.Audio(path_selection))

                #visualize with numpy array
                sampling_freq, array = wavfile.read(path_selection)
                print("sampling_freq:", sampling_freq)

                duration = len(array) / sampling_freq
                print("duration:", duration)

                duration = np.linspace(0,duration, len(array))
                plt.figure(figsize=(20,10))
                plt.plot(duration,array)
                plt.title(selection)
                plt.xlabel("Duration (s)")
                plt.ylabel("Magnitude")
                

interactive(children=(Dropdown(description='machine', options=('bearing', 'fan', 'gearbox', 'slider', 'ToyCar'…

In [5]:
IPython.display.Audio(path_selection)

NameError: name 'path_selection' is not defined

In [ ]:
example = os.path.join(bearing,"section_00_source_train_normal_0001_pro_A_vel_4_loc_A.wav")
IPython.display.Audio(example)

NameError: name 'bearing' is not defined